<a href="https://colab.research.google.com/github/RodolfoMontes/SmarttBot-Spark/blob/main/Sarmttbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# instalar as dependências
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [3]:
import findspark
findspark.init()

## [SparkSession](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.SparkSession.html)

O ponto de entrada para programar o Spark com a API Dataset e DataFrame.

Uma SparkSession pode ser utilizada para criar DataFrames, registrar DataFrames como tabelas, executar consultas SQL em tabelas, armazenar em cache e ler arquivos parquet. Para criar uma SparkSession, use o seguinte padrão de construtor:

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

In [5]:
spark

[Site ngrok](https://ngrok.com)

In [6]:
!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [7]:
get_ipython().system_raw('./ngrok http 4050 &')

In [8]:
!curl -s http://localhost:4040/api/tunnels

{"tunnels":[{"name":"command_line (http)","uri":"/api/tunnels/command_line%20%28http%29","public_url":"http://d261-34-66-69-106.ngrok.io","proto":"http","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}},{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://d261-34-66-69-106.ngrok.io","proto":"https","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}


In [9]:
spark

### Montando nosso drive

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Carregando os dados

In [25]:
person_person = spark.read.options(header=True).csv('/content/drive/MyDrive/SmarttBot-Spark/Person_Person.csv', sep=';', inferSchema=True)
production_product = spark.read.options(header=True).csv('/content/drive/MyDrive/SmarttBot-Spark/Production_Product.csv', sep=';', inferSchema=True)
sales_customer = spark.read.options(header=True).csv('/content/drive/MyDrive/SmarttBot-Spark/Sales_Customer.csv', sep=';', inferSchema=True)
sales_salesorderdetail = spark.read.options(header=True).csv('/content/drive/MyDrive/SmarttBot-Spark/Sales_SalesOrderDetail.csv', sep=';', inferSchema=True)
sales_salesorderheader = spark.read.options(header=True).csv('/content/drive/MyDrive/SmarttBot-Spark/Sales_SalesOrderHeader.csv', sep=';', inferSchema=True)
sales_specialofferproduct = spark.read.options(header=True).csv('/content/drive/MyDrive/SmarttBot-Spark/Sales_SpecialOfferProduct.csv', sep=';', inferSchema=True)

##1 - Escreva uma query que retorna a quantidade de linhas na tabela Sales.SalesOrderDetail pelo campo SalesOrderID, desde que tenham pelo menos três linhas de detalhes.

In [35]:
person_person.limit(5).toPandas()

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,0,NULL,Ken,J,Sánchez,NULL,0,NULL,"""<IndividualSurvey xmlns=""""http://schemas.micr...",92C4279F-1207-48A3-8448-4636514EB7E2,2009-01-07 00:00:00.000
1,2,EM,0,NULL,Terri,Lee,Duffy,NULL,1,NULL,"""<IndividualSurvey xmlns=""""http://schemas.micr...",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24 00:00:00.000
2,3,EM,0,NULL,Roberto,NULL,Tamburello,NULL,0,NULL,"""<IndividualSurvey xmlns=""""http://schemas.micr...",E1A2555E-0828-434B-A33B-6F38136A37DE,2007-11-04 00:00:00.000
3,4,EM,0,NULL,Rob,NULL,Walters,NULL,0,NULL,"""<IndividualSurvey xmlns=""""http://schemas.micr...",F2D7CE06-38B3-4357-805B-F4B6B71C01FF,2007-11-28 00:00:00.000
4,5,EM,0,Ms.,Gail,A,Erickson,NULL,0,NULL,"""<IndividualSurvey xmlns=""""http://schemas.micr...",F3A3F6B4-AE3B-430C-A754-9F2231BA6FEF,2007-12-30 00:00:00.000


In [36]:
person_person.printSchema()

root
 |-- BusinessEntityID: integer (nullable = true)
 |-- PersonType: string (nullable = true)
 |-- NameStyle: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- FirstName: string (nullable = true)
 |-- MiddleName: string (nullable = true)
 |-- LastName: string (nullable = true)
 |-- Suffix: string (nullable = true)
 |-- EmailPromotion: integer (nullable = true)
 |-- AdditionalContactInfo: string (nullable = true)
 |-- Demographics: string (nullable = true)
 |-- rowguid: string (nullable = true)
 |-- ModifiedDate: string (nullable = true)



In [37]:
production_product.printSchema()

root
 |-- ProductID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- ProductNumber: string (nullable = true)
 |-- MakeFlag: integer (nullable = true)
 |-- FinishedGoodsFlag: integer (nullable = true)
 |-- Color: string (nullable = true)
 |-- SafetyStockLevel: integer (nullable = true)
 |-- ReorderPoint: integer (nullable = true)
 |-- StandardCost: string (nullable = true)
 |-- ListPrice: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- SizeUnitMeasureCode: string (nullable = true)
 |-- WeightUnitMeasureCode: string (nullable = true)
 |-- Weight: string (nullable = true)
 |-- DaysToManufacture: integer (nullable = true)
 |-- ProductLine: string (nullable = true)
 |-- Class: string (nullable = true)
 |-- Style: string (nullable = true)
 |-- ProductSubcategoryID: string (nullable = true)
 |-- ProductModelID: string (nullable = true)
 |-- SellStartDate: string (nullable = true)
 |-- SellEndDate: string (nullable = true)
 |-- DiscontinuedDate: string (nul